### Agent-Lab: Multi-Agent -> Browser

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [ ]:
%%capture

import json
import os

os.chdir("..")

from IPython.display import Markdown, display

from dotenv import load_dotenv

load_dotenv()

from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest

# graph rendering
import nest_asyncio

nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### Anthropic Browser

In [ ]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-sonnet-4-0", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_browser_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_browser_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [ ]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="Using the browser, visit https://agent-lab.btech.software/docs and describe the endpoints of the REST API",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_browser_agent.get_input_params(message, schema="public")
config = anthropic_browser_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_browser_agent.format_response(result)

In [ ]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

In [ ]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

---
### OpenAI Browser

In [ ]:
# Create Workflow
openai_agent = experiment_utils.create_openai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="gpt-5-nano", api_key=os.getenv("OPENAI_API_KEY")
)
openai_browser_agent = container.coordinator_planner_supervisor_agent()
openai_workflow_builder = openai_browser_agent.get_workflow_builder(openai_agent["id"])
openai_workflow = openai_workflow_builder.compile(checkpointer=checkpointer)

In [ ]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="Using the browser, visit https://agent-lab.btech.software/docs and describe the endpoints of the REST API",
    agent_id=openai_agent["id"],
)

inputs = openai_browser_agent.get_input_params(message, schema="public")
config = openai_browser_agent.get_config(openai_agent["id"])
result = openai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = openai_browser_agent.format_response(result)

In [ ]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

In [ ]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))